<a href="https://colab.research.google.com/github/VectorInstitute/Causal_Inference_Laboratory/blob/main/notebooks/Demo_DeepLearning_DoubleML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Inference via Deep Learning and Double Machine Learning

In this tutorial, we will demo causal inference (CI) based on deep learning (DL) models and other models tailored for CI, including
- Treatment-Agnostic Representation Network (TAR-Net) [1]
- Dragon-Net [2]
- Double Machine Learning [3]



[1] U. Shalit, F. D. Johansson, and D. Sontag, “Estimating individual treatment effect: generalization bounds and algorithms,” in ICML 2017.

[2] C. Shi, D. Blei, and V. Veitch, “Adapting neural networks for the estimation of treatment effects,” in NeurIPS 2019.

[3] V. Chernozhukov, D. Chetverikov, M. Demirer, E. Duflo, C. Hansen, W. Newey, and J. Robins, ”Double/debiased machine learning for treatment and structural parameters,” The Econometrics Journal, vol. 21, no. 1, pp. 1–68, Jan 2018.

## Preparation
We will use some modules from our public repo on https://github.com/VectorInstitute/Causal_Inference_Laboratory.git

In [8]:
!git clone https://github.com/VectorInstitute/Causal_Inference_Laboratory.git
!mv Causal_Inference_Laboratory code
!mv code/data ./data
!mv code/utils ./utils
!mv code/models ./models
!mv code/estimation_results ./estimation_results

Cloning into 'Causal_Inference_Laboratory'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 489 (delta 90), reused 84 (delta 40), pack-reused 333
Receiving objects: 100% (489/489), 27.99 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (229/229), done.
mv: cannot move 'Causal_Inference_Laboratory' to 'code/Causal_Inference_Laboratory': Directory not empty
mv: cannot stat 'code/data': No such file or directory
mv: cannot stat 'code/utils': No such file or directory
mv: cannot stat 'code/models': No such file or directory
mv: cannot stat 'code/estimation_results': No such file or directory


In [9]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import time

from typing import Dict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers, regularizers



import utils.preprocessing as helper
from utils.preprocessing import sys_config

## Dataset
In this demo, we will use the Infant Health and Development Program (IHDP) dataset [4].

- from a randomized experiment studying the effect of home visits by specialists on future cognitive test scores of children;
- the children of non-white mothers in the treated set are removed to de-randomize the experiment;
- each unit is simulated for a treated and a control outcome;

Details:
- the train split (672 units for each realization) and the test split (75 units for each realization) 
- x: covariates;
- t: treatment;
- yf: factual outcome;
- ycf: counterfactual outcome;
- mu0: noiseless potential control outcome;
- mu1: noiseless potential treated outcome.



*IHDP-100/1000 contains 100/1000 realizations. In our demo, we will just use the first 5 realizations of IHDP-100.*

[4] J. L. Hill, “Bayesian nonparametric modeling for causal inference,” Journal of Computational and Graphical Statistics, vol. 20, no. 1, pp. 217–240, 2011. [Online]. Available: https://doi.org/10.1198/jcgs.2010.08162

In [10]:
datasets_folder = sys_config["datasets_folder"]
results_folder = sys_config["results_folder"]

dataset_name = "IHDP-100" 
num_realizations = 5

x_train_all, t_train_all, yf_train_all = helper.load_IHDP_observational(
        datasets_folder, dataset_name, details=True
    )
x_test_all, t_test_all, yf_test_all = helper.load_IHDP_out_of_sample(
    datasets_folder, dataset_name, details=True
)

x_train, t_train, yf_train = x_train_all[:, :, :num_realizations], t_train_all[:, :num_realizations], yf_train_all[:, :num_realizations]
x_test = x_test_all[:, :, :num_realizations]
print(x_train[0, :, 0])

-------------------------------------------------------------------------------
The details of the train split of IHDP-100 dataset:
Number of realizations: 100
ate () int64
mu1 (672, 100) float64
mu0 (672, 100) float64
yadd () int64
yf (672, 100) float64
ycf (672, 100) float64
t (672, 100) float64
x (672, 25, 100) float64
ymul () int64
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
The details of the test split of IHDP-100 dataset:
Number of realizations: 100
ate () int64
mu1 (75, 100) float64
mu0 (75, 100) float64
yadd () int64
yf (75, 100) float64
ycf (75, 100) float64
t (75, 100) float64
x (75, 25, 100) float64
ymul () int64
-------------------------------------------------------------------------------
[ 0.31658816  0.59658219 -0.36089799 -0.87960599  0.37108604 -1.18901798
  1.          0.          1.          0.          1.          0.
  0.          2.          0.     

## Configuration

In [11]:
# define neural network architectures
class EpsilonLayer(layers.Layer):
    def __init__(self):
        super(EpsilonLayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.epsilon = self.add_weight(
            name="epsilon",
            shape=[1, 1],
            initializer="RandomNormal",
            trainable=True,
        )
        super(EpsilonLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        return self.epsilon * tf.ones_like(inputs)[:, 0:1]


def make_dragonnet(input_dim, reg_l2):
    """
    Create the dragonnet which has three heads.
    :param input_dim:
    :param reg_l2:
    :return:
    """
    inputs = keras.Input(shape=(input_dim,), name="input")

    # representation
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(inputs)
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(x)
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(x)

    t_predictions = layers.Dense(units=1, activation="sigmoid")(x)

    # two heads for two hypotheses
    y0_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(x)
    y1_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(x)

    # second layer
    y0_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(y0_hidden)
    y1_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(y1_hidden)

    # third layer
    y0_predictions = layers.Dense(
        units=1, kernel_regularizer=regularizers.L2(reg_l2), name="y0_pred"
    )(y0_hidden)
    y1_predictions = layers.Dense(
        units=1, kernel_regularizer=regularizers.L2(reg_l2), name="y1_pred"
    )(y1_hidden)

    dl = EpsilonLayer()
    epsilons = dl(t_predictions, name="epsilon")
    concat_pred = layers.Concatenate(axis=1)(
        [y0_predictions, y1_predictions, t_predictions, epsilons]
    )
    model = keras.Model(inputs=inputs, outputs=concat_pred, name="dragonnet")
    return model


def make_tarnet(input_dim, reg_l2):
    """
    Create the TARNet which has three heads.
    :param input_dim:
    :param reg_l2:
    :return:
    """
    inputs = keras.Input(shape=(input_dim,), name="input")

    # representation
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(inputs)
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(x)
    x = layers.Dense(
        units=200, activation="elu", kernel_initializer="RandomNormal"
    )(x)

    # different from Dragonnet, here TARNet directly uses inputs to predict t
    t_predictions = layers.Dense(units=1, activation="sigmoid")(inputs)

    # two heads for two hypotheses
    y0_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(x)
    y1_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(x)

    # second layer
    y0_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(y0_hidden)
    y1_hidden = layers.Dense(
        units=100, activation="elu", kernel_regularizer=regularizers.L2(reg_l2)
    )(y1_hidden)

    # third layer
    y0_predictions = layers.Dense(
        units=1, kernel_regularizer=regularizers.L2(reg_l2), name="y0_pred"
    )(y0_hidden)
    y1_predictions = layers.Dense(
        units=1, kernel_regularizer=regularizers.L2(reg_l2), name="y1_pred"
    )(y1_hidden)

    dl = EpsilonLayer()
    epsilons = dl(t_predictions, name="epsilon")
    concat_pred = layers.Concatenate(axis=1)(
        [y0_predictions, y1_predictions, t_predictions, epsilons]
    )
    model = keras.Model(inputs=inputs, outputs=concat_pred, name="tarnet")
    return model


# define customized loss functions
def regression_loss(concat_true, concat_pred):
    # concat_true: [y, t]
    # concat_pred: [y0_predictions, y1_predictions, t_predictions, epsilons]
    y_true = concat_true[:, 0]
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    # if treatment is 0, compare the factual y (y0) and y0_pred
    loss0 = tf.reduce_sum((1.0 - t_true) * tf.square(y_true - y0_pred))

    # if treatment is 1, compare the factual y (y1) and y1_pred
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    return loss0 + loss1

def binary_classification_loss(concat_true, concat_pred):
    t_true = concat_true[:, 1]
    t_pred = concat_pred[:, 2]
    t_pred = (t_pred + 0.001) / 1.002
    bce = tf.keras.losses.BinaryCrossentropy()
    losst = tf.reduce_sum(bce(t_true, t_pred))
    return losst

def dragonnet_loss_binarycross(concat_true, concat_pred):
    return regression_loss(
        concat_true, concat_pred
    ) + binary_classification_loss(concat_true, concat_pred)

def make_tarreg_loss(ratio=1.0, dragonnet_loss=dragonnet_loss_binarycross):
    def tarreg_ATE_unbounded_domain_loss(concat_true, concat_pred):
        vanilla_loss = dragonnet_loss(concat_true, concat_pred)

        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]

        y0_pred = concat_pred[:, 0]
        y1_pred = concat_pred[:, 1]
        t_pred = concat_pred[:, 2]

        epsilons = concat_pred[:, 3]
        t_pred = (t_pred + 0.01) / 1.02

        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred

        h = t_true / t_pred - (1 - t_true) / (1 - t_pred)

        y_pert = y_pred + epsilons * h
        targeted_regularization = tf.reduce_sum(tf.square(y_true - y_pert))

        loss = vanilla_loss + ratio * targeted_regularization
        return loss

    return tarreg_ATE_unbounded_domain_loss
    
# define other metrics
def treatment_accuracy(concat_true, concat_pred):
    t_true = concat_true[:, 1]
    t_pred = concat_pred[:, 2]
    return tf.keras.metrics.binary_accuracy(t_true, t_pred)

def track_epsilon(concat_true, concat_pred):
    epsilons = concat_pred[:, 3]
    return tf.abs(tf.reduce_mean(epsilons))

## Estimation via TARNet and Dragonnet

In [12]:
def train_and_estimate_Dragonnet(x, t, y_unscaled, x_test, estimator_name):
    """
    Training using Drogonnet.
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """

    reg_l2 = 0.01
    batch_size = 32

    # 6 cont. features + 19 discrete features => 19 discrete + 6 cont. features
    num_features = x.shape[1]
    if num_features == 25:  # IHDP
        perm = np.arange(6, 25).tolist() + np.arange(6).tolist()
        x = x[:, perm]
        x_test = x_test[:, perm]
    t = t[:, None]  # add a dummy axis (from original dragonnet code)
    y_unscaled = y_unscaled[:, None]

    # y_scaler = StandardScaler().fit(y_unscaled)  # from dragonnet code
    # y = y_scaler.transform(y_unscaled) # normalize y from original code
    y = y_unscaled
    if estimator_name == "Dragonnet":
        model = make_dragonnet(x.shape[-1], reg_l2)
    elif estimator_name == "TARNet":
        model = make_tarnet(x.shape[-1], reg_l2)

    metrics = [
        regression_loss,
        binary_classification_loss,
        treatment_accuracy,
        track_epsilon,
    ]

    loss = make_tarreg_loss(
        ratio=1.0, dragonnet_loss=dragonnet_loss_binarycross
    )

    yt_train = np.concatenate([y, t], axis=1)
    start_time = time.time()


    # from original code, training in two phasesL use a high learning rate first, 
    # then use a low learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=loss,
        metrics=metrics,
    )
    
    verbose = 0
    adam_callbacks = [
        keras.callbacks.TerminateOnNaN(),
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=2, min_delta=0.0
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="loss",
            factor=0.5,
            patience=5,
            verbose=verbose,
            mode="auto",
            min_delta=1e-8,
            cooldown=0,
            min_lr=0,
        ),
    ]
    
    model.fit(
        x,
        yt_train,
        callbacks=adam_callbacks,
        validation_split=0.2,
        epochs=100,
        batch_size=batch_size,
        verbose=verbose,
    )


    sgd_callbacks = [
        keras.callbacks.TerminateOnNaN(),
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=40, min_delta=0.0
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="loss",
            factor=0.5,
            patience=5,
            verbose=verbose,
            mode="auto",
            min_delta=0.0,
            cooldown=0,
            min_lr=0,
        ),
    ]


    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=1e-5, momentum=0.9, nesterov=True
        ),
        loss=loss,
        metrics=metrics,
    )

    model.fit(
        x,
        yt_train,
        callbacks=sgd_callbacks,
        validation_split=0.2,
        epochs=300,
        batch_size=batch_size,
        verbose=verbose,
    )


    elapsed_time = time.time() - start_time
    text = f"Elapsed_time is: {elapsed_time:.4f} seconds"
    print(f"{text:-^79}")

    yt_hat_test = model.predict(x_test)
    yt_hat_train = model.predict(x)
    # y0_in, y1_in = _split_output(yt_hat_train, t, y_scaler, 'in')
    # y0_out, y1_out = _split_output(yt_hat_test, None, y_scaler, 'out')
    y0_in, y1_in = yt_hat_train[:, 0], yt_hat_train[:, 1]
    ate_in = np.mean(y1_in - y0_in)
    y0_out, y1_out = yt_hat_test[:, 0], yt_hat_test[:, 1]
    ate_out = np.mean(y1_out - y0_out)

    # propensity_in = yt_hat_train[:, 2]
    # propensity_out = yt_hat_test[:, 2]

    tf.keras.backend.clear_session()
    # return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out, propensity_in, propensity_out
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out



In [13]:
estimator_set = ["TARNet", "Dragonnet"]
for estimator_name in estimator_set:
        y0_in_all, y1_in_all, y0_out_all, y1_out_all = [], [], [], []
        ate_in_all, ate_out_all = [], []
        for i in range(num_realizations):
            text = f" Estimation of realization {i} via {estimator_name}"
            print(f"{text:-^79}")
            x, t, yf = x_train[:, :, i], t_train[:, i], yf_train[:, i]
            x_t = x_test[:, :, i]
            # train the estimator and predict for this realization
            (
                y0_in,
                y1_in,
                ate_in,
                y0_out,
                y1_out,
                ate_out,
            ) = train_and_estimate_Dragonnet(x, t, yf, x_t, estimator_name)
            y0_in_all.append(y0_in)
            y1_in_all.append(y1_in)
            ate_in_all.append(ate_in)
            y0_out_all.append(y0_out)
            y1_out_all.append(y1_out)
            ate_out_all.append(ate_out)
        # follow the dimension order of the dataset,
        # i.e., realizations are captured by the last index
        y0_in_all = np.squeeze(np.array(y0_in_all).transpose())
        y1_in_all = np.squeeze(np.array(y1_in_all).transpose())
        y0_out_all = np.squeeze(np.array(y0_out_all).transpose())
        y1_out_all = np.squeeze(np.array(y1_out_all).transpose())
        ate_in_all = np.array(ate_in_all)
        ate_out_all = np.array(ate_out_all)

        # save estimation results
        estimation_result_folder = os.path.join(
            results_folder, dataset_name, estimator_name
        )
        print(f"Saving {estimation_result_folder}.")
        helper.save_in_and_out_results(
            estimation_result_folder,
            y0_in_all,
            y1_in_all,
            ate_in_all,
            y0_out_all,
            y1_out_all,
            ate_out_all,
        )



-------------------- Estimation of realization 0 via TARNet--------------------
-----------------------Elapsed_time is: 19.2062 seconds------------------------
21/21 [==============================] - 0s 2ms/step
-------------------- Estimation of realization 1 via TARNet--------------------
-----------------------Elapsed_time is: 26.2874 seconds------------------------
21/21 [==============================] - 0s 2ms/step
-------------------- Estimation of realization 2 via TARNet--------------------
-----------------------Elapsed_time is: 20.0839 seconds------------------------
21/21 [==============================] - 0s 2ms/step
-------------------- Estimation of realization 3 via TARNet--------------------
-----------------------Elapsed_time is: 25.1130 seconds------------------------
21/21 [==============================] - 0s 2ms/step
-------------------- Estimation of realization 4 via TARNet--------------------
-----------------------Elapsed_time is: 17.3324 seconds-------------

## Estimation via Double Machine Learning

In [27]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge

def train_and_estimate_DML_helper(x_aux, x_new, t_aux, t_new, yf_aux, yf_new):
    # use auxiliary data to fit t from x and y from x
    LR = LogisticRegression()
    LR.fit(x_aux, t_aux)

    LS = Ridge()
    LS.fit(x_aux, yf_aux)

    # get residuals 
    t_r = t_new - LR.predict_proba(x_new)[:, 1]
    y_r = yf_new - LS.predict(x_new)

    OLS = LinearRegression()
    OLS.fit(t_r.reshape(-1, 1), y_r)  # single feature

    ate_in, ate_out = OLS.coef_.item(), OLS.coef_.item()
    return ate_in, ate_out

def train_and_estimate_DML(x, t, yf, x_test):
    """
    Training using double machine learning (DML), i.e., R-Learner.
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """
    x_aux, x_new, t_aux, t_new, yf_aux, yf_new = train_test_split(
        x, t, yf, test_size=0.5, shuffle=False
    )

    ate_in_1, ate_out_1 = train_and_estimate_DML_helper(x_aux, x_new, t_aux, t_new, yf_aux, yf_new)
    ate_in_2, ate_out_2 = train_and_estimate_DML_helper(x_new, x_aux, t_new, t_aux, yf_new, yf_aux)

    return (ate_in_1+ate_in_2)/2.0, (ate_out_1+ate_out_2)/2.0


estimator_set = ["DML"]
for estimator_name in estimator_set:
    ate_in_all = []
    ate_out_all = []
    for i in range(num_realizations):
        text = f" Estimation of realization {i} via {estimator_name}"
        print(f"{text:-^79}")
        x, t, yf = x_train[:, :, i], t_train[:, i], yf_train[:, i]
        x_t = x_test[:, :, i]
        ate_in, ate_out = train_and_estimate_DML(x, t, yf, x_t)
        ate_in_all.append(ate_in)
        ate_out_all.append(ate_out)

    y0_in_all = None
    y1_in_all = None
    y0_out_all = None
    y1_out_all = None
    ate_in_all = np.array(ate_in_all)
    ate_out_all = np.array(ate_in_all)
    # save estimation results
    estimation_result_folder = os.path.join(
        results_folder, dataset_name, estimator_name
    )
    print(f"Saving {estimation_result_folder}.")
    helper.save_in_and_out_results(
        estimation_result_folder,
        y0_in_all,
        y1_in_all,
        ate_in_all,
        y0_out_all,
        y1_out_all,
        ate_out_all,
    )


--------------------- Estimation of realization 0 via DML----------------------
--------------------- Estimation of realization 1 via DML----------------------
--------------------- Estimation of realization 2 via DML----------------------
--------------------- Estimation of realization 3 via DML----------------------
--------------------- Estimation of realization 4 via DML----------------------
Saving ./estimation_results/IHDP-100/DML.


## Evaluation

In [28]:
def calculate_mae_ATE(y0, y1, mu0, mu1):
    """
    Calculate the mean absolute error of ATE estimation
    :param y0: estimation of y0
    :param y1: estimation of y1
    :param mu0: ground-truth of potential outcome of T = 0
    :param mu1: ground-truth of potential outcome of T = 1
    :return:
    """
    ATE_gt = np.mean(mu1 - mu0)
    ATE_pred = np.mean(y1 - y0)
    return np.abs(ATE_gt - ATE_pred)


def calculate_mae_ATEs(y0, y1, mu0, mu1):
    """
    Calculate the mean absolute error of ATE estimation of
    multiple realizations/datasets
    :param y0: estimation of y0 of multiple realizations
    :param y1: estimation of y1 of multiple realizations
    :param mu0: ground-truth of y0 of multiple realizations
    :param mu1: ground-truth of y1 of multiple realizations
    :return: ndarray of MAE of ATE of all realizations
    """
    assert y0.shape == mu0.shape, f"shape of y0 and mu0: {y0.shape}{mu0.shape}"
    assert y1.shape == mu1.shape, f"shape of y1 and mu1: {y1.shape}{mu1.shape}"
    assert y0.shape == y1.shape, f"shape of y0 and y1: {y0.shape}{y1.shape}"
    num_real = y0.shape[-1]
    mae_ATE = np.zeros(num_real)
    for i in range(num_real):
        mae_ATE[i] = calculate_mae_ATE(
            y0[:, i], y1[:, i], mu0[:, i], mu1[:, i]
        )
    return mae_ATE

def calculate_mae_ATEs_scalar(y0, y1, ate):
    """
    Calculate the mean absolute error of ATE estimation of
    multiple realizations/datasets
    :param y0: estimation of y0 of multiple realizations
    :param y1: estimation of y1 of multiple realizations
    :param ate: ground-truth ate of multiple realizations
    :return: ndarray of MAE of ATE of all realizations
    """
    assert y0.shape == y1.shape, f"shape of y0 and y1: {y0.shape}{y1.shape}"
    num_realizations = y0.shape[-1]
    # each realization has an ate
    if np.isscalar(ate) or ate.shape == (1,):
        ate = ate * np.ones(num_realizations)
    elif ate.shape == (1, 1):
        ate = ate.item() * np.ones(num_realizations)
    mae_ATE = np.zeros(num_realizations)
    for i in range(num_realizations):
        mae_ATE[i] = np.abs(np.mean(y1[:, i] - y0[:, i]) - ate[i])
    return mae_ATE

def calculate_PEHE(y0, y1, mu0, mu1):
    """
    Calculate the Precision Estimation of Heterogeneous Effect (PEHE) of
    one realization/dataset
    :param y0: estimation of y0
    :param y1: estimation of y1
    :param mu0: ground-truth of potential outcome of T = 0
    :param mu1: ground-truth of potential outcome of T = 1
    :return:
    """
    return np.sqrt(mean_squared_error(mu1 - mu0, y1 - y0))


def calculate_PEHEs(y0, y1, mu0, mu1):
    """
    Calculate the Precision Estimation of Heterogeneous Effect (PEHE) of
    multiple realizations/datasets
    :param y0: estimation of y0 of multiple realizations
    :param y1: estimation of y1 of multiple realizations
    :param mu0: ground-truth of y0 of multiple realizations
    :param mu1: ground-truth of y1 of multiple realizations
    """
    assert y0.shape == mu0.shape, f"shape of y0 and mu0: {y0.shape}{mu0.shape}"
    assert y1.shape == mu1.shape, f"shape of y1 and mu1: {y1.shape}{mu1.shape}"
    assert y0.shape == y1.shape, f"shape of y0 and y1: {y0.shape}{y1.shape}"
    num_realizations = y0.shape[-1]
    pehe = np.zeros(num_realizations)
    for i in range(num_realizations):
        pehe[i] = calculate_PEHE(y0[:, i], y1[:, i], mu0[:, i], mu1[:, i])
    return pehe


def calculate_metrics(y0, y1, ate, mu0, mu1, ate_gt, metric="PEHE"):
    if metric == "PEHE":
        metric_over_realizations = calculate_PEHEs(y0, y1, mu0, mu1)
    elif metric == "MAE":
        if y0 is None or (y0.size == 1 and y0.item() == None):
            metric_over_realizations = calculate_mae_ATEs_scalar(mu0, mu1, ate)
        else:
            metric_over_realizations = calculate_mae_ATEs(y0, y1, mu0, mu1)
    return metric_over_realizations

In [29]:
dataset_name = "IHDP-100"
estimator_set = [
    "Dragonnet",
    "TARNet",
    "DML",
]
metrics_set = ["PEHE", "MAE"]
num_realizations = 5
print(f'{" Evaluation ":-^79}')
results_in: Dict[str, Dict] = {}
results_out: Dict[str, Dict] = {}

mu0_in, mu1_in, mu0_out, mu1_out = helper.load_IHDP_ground_truth(
    datasets_folder, dataset_name, details=False
)
mu0_in, mu1_in =  mu0_in[:, :num_realizations], mu1_in[:, :num_realizations]
mu0_out, mu1_out = mu0_out[:, :num_realizations], mu1_out[:, :num_realizations]
ate_in_gt = np.mean(mu1_in - mu0_in)
ate_out_gt = np.mean(mu1_out - mu0_out)

for estimator_name in estimator_set:
    estimation_result_folder = os.path.join(
        results_folder, dataset_name, estimator_name
    )
    (
        y0_in,
        y1_in,
        ate_in,
        y0_out,
        y1_out,
        ate_out,
    ) = helper.load_in_and_out_results(estimation_result_folder)
    results_in[estimator_name] = {}
    results_out[estimator_name] = {}
    for metric in metrics_set:
        if estimator_name == "DML" and metric == "PEHE":
            results_in[estimator_name][metric] = {"mean": "N/A"}
            results_out[estimator_name][metric] = {"mean": "N/A"}
            continue

        metric_in = calculate_metrics(
            y0_in, y1_in, ate_in, mu0_in, mu1_in, ate_in_gt, metric=metric
        )

        results_in[estimator_name][metric] = {
            "mean": np.mean(metric_in),
        }

        if estimator_name == "DML" and metric == "PEHE":
            results_out[estimator_name][metric] = {
                "mean": "N/A"
            }
            continue

        metric_out = calculate_metrics(
            y0_out,
            y1_out,
            ate_out,
            mu0_out,
            mu1_out,
            ate_out_gt,
            metric=metric,
        )

        results_out[estimator_name][metric] = {
            "mean": np.mean(metric_out)
        }

print(f'{" In-sample results ":-^79}')
for metric in metrics_set:
    for estimator_name in estimator_set:
        print(metric, estimator_name, results_in[estimator_name][metric])
print(f'{" Out-of-sample results ":-^79}')
for metric in metrics_set:
    for estimator_name in estimator_set:
        print(metric, estimator_name, results_out[estimator_name][metric])

--------------------------------- Evaluation ----------------------------------
------------------------------ In-sample results ------------------------------
PEHE Dragonnet {'mean': 0.6218022373142205}
PEHE TARNet {'mean': 0.6401338339092958}
PEHE DML {'mean': 'N/A'}
MAE Dragonnet {'mean': 0.11109651623225555}
MAE TARNet {'mean': 0.13775950879437798}
MAE DML {'mean': 0.19793177791431624}
---------------------------- Out-of-sample results ----------------------------
PEHE Dragonnet {'mean': 0.6042200734424676}
PEHE TARNet {'mean': 0.627264889527299}
PEHE DML {'mean': 'N/A'}
MAE Dragonnet {'mean': 0.1323210635143779}
MAE TARNet {'mean': 0.16186412953915816}
MAE DML {'mean': 0.17733408027474057}
